In [1]:
import pandas as pd

import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import re

In [2]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

# url = input('Enter - ')
url = 'https://www.cricbuzz.com/cricket-series/ipl-2021/auction/completed'


req = urllib.request.Request(url, headers=hdr)
page = urllib.request.urlopen(req, context=ctx)
html = page.read()

soup = BeautifulSoup(html, 'html.parser')

lst = list()
tags = soup('a')

raw_data = list()
link="cb-col cb-col-100 cb-font-14 text-center"

div = soup.find_all('a', {"class" : link})
for d in div:
    raw_data.append(re.split(r'\s{2,}', d.text))
    
print(raw_data[0])

['', 'Arjun Tendulkar', 'Bowling Allrounder • India', 'SOLD', 'Base Price 20.00', 'Lakh', 'Final Price 20.00', 'Lakh', 'Sold To MI', '']


In [3]:
name = list()
speciality = list()
country = list()
status = list()
base_price = list()
final_price = list()
team = list()

for plyr in raw_data:
    x = plyr[2].split("•")
    if len(x) > 1: 
        name.append(plyr[1])
        speciality.append(x[0])
        country.append(x[1])

        bs = int(float(plyr[4].split(" ")[2]))
        if plyr[5] == "Crore": bs *= 100
        base_price.append(bs)
         
        status.append(plyr[3])
        if plyr[3] == 'SOLD':
            fp = int(float(plyr[6].split(" ")[2]))
            if plyr[7] == "Crore": fp *= 100
            final_price.append(fp)    
            team.append(plyr[8].split()[2])
        else:
            final_price.append('-')
            team.append('-')

In [4]:
df = pd.DataFrame({'Name': name,
                   'Role': speciality,
                   'Country': country,
                   'Status': status,
                   'Base_Price(Lakh)': base_price,
                   'Final_Price(Lakh)': final_price,
                   'Team': team})

df.to_csv('auction.csv')